In [1]:
import pandas as pd
import re

In [2]:
netflix_data = pd.read_csv('../data/raw/netflix_branded_titles_with_release_year.csv')

In [3]:
# Read in the data
imdb_data = pd.read_csv('../data/raw/title.basics.tsv', sep='\t')

C:\Users\Joeyz\AppData\Local\Temp\ipykernel_15268\818661964.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_data = pd.read_csv('../data/raw/title.basics.tsv', sep='\t')


In [4]:
imdb_data.columns = ['tconst', 'titleType', 'title', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']

In [7]:
part1 = imdb_data[imdb_data['startYear']!='\\N']
part2 = part1[part1['startYear'].astype('int')>=2020]
part3 = part2[part2['titleType'].isin(['movie', 'tvSeries'])]

In [8]:
part3['title'] = part3['title'].str.lower()

C:\Users\Joeyz\AppData\Local\Temp\ipykernel_15268\1917613317.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part3['title'] = part3['title'].str.lower()


In [17]:
part2.to_csv('../data/processed/imdb_data_20_to_22.csv', index=False)

In [25]:
netflix_data.columns

Index(['title_desc', 'release_year', 'release_date', 'is_top10'], dtype='object')

In [9]:
#split combined_title into title and season
netflix_data.rename(columns={'title_desc':'combined_title'}, inplace=True)
netflix_data[['title', 'season']] = netflix_data['combined_title'].str.split(': Season ', expand=True)

In [10]:
netflix_data['title'] = netflix_data['title'].str.lower()

In [11]:
netflix_data['is_tv_show'] = netflix_data['season'].notnull()

In [12]:
netflix_data['release_year'] = netflix_data['release_year'].astype('int')
part3['startYear'] = part3['startYear'].astype('int')

C:\Users\Joeyz\AppData\Local\Temp\ipykernel_15268\2947123683.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part3['startYear'] = part3['startYear'].astype('int')


In [16]:
merged = netflix_data[~netflix_data['is_tv_show']].merge(part3, how='left', left_on=['title', 'release_year'], right_on=['title', 'startYear'], indicator=True)

In [17]:
merged[merged['_merge']=='left_only']

,combined_title,release_year,release_date,is_top10,title,season,is_tv_show,tconst,titleType,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,_merge
0,10 Days With Santa Claus,2020,20201204,False,10 days with santa claus,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
8,37 Seconds,2020,20200131,False,37 seconds,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
11,7SEEDS: Part 2,2020,20200326,True,7seeds: part 2,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
12,8,2020,20200612,False,8,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
13,800 Meters: Limited Series,2022,20220325,True,800 meters: limited series,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1136,Yuri Marçal: Honest Mistake,2022,20220602,False,yuri marçal: honest mistake,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1139,jeen-yuhs: A Kanye Trilogy,2022,20220216,True,jeen-yuhs: a kanye trilogy,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1140,"tick, tick...BOOM!",2021,20211119,True,"tick, tick...boom!",None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1141,Òlòtūré,2020,20201002,False,òlòtūré,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [19]:
netflix_data[netflix_data['title']=='8']

,combined_title,release_year,release_date,is_top10,title,season,is_tv_show
19,8,2020,20200612,False,8,None,False


In [25]:
merged2['titleType'].value_counts()

movie           613
tvSeries        604
tvEpisode       483
short           323
tvSpecial       114
tvMiniSeries     69
video            21
tvMovie          21
tvShort           3
videoGame         1
Name: titleType, dtype: int64

In [30]:
merged2[merged2['titleType']=='video']

,combined_title,release_year,release_date,is_top10,title,season,is_tv_show,tconst,titleType,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
115,All Together Now,2020,20200828,False,all together now,None,False,tt13905548,video,All Together Now,0,2020.0,\N,4,"Horror,Short"
196,Another Life: Season 2,2021,20211014,True,another life,2,True,tt15444526,video,Another Life,0,2021.0,\N,\N,"Fantasy,Short"
259,Baby: Season 3,2020,20200916,True,baby,3,True,tt13059214,video,Baby,0,2020.0,\N,10,"Music,Short"
262,Baby: Season 3,2020,20200916,True,baby,3,True,tt15686934,video,Baby,0,2020.0,\N,\N,Short
408,Can You Hear Me?: Season 2,2020,20200113,False,can you hear me?,2,True,tt16403266,video,Can You Hear Me?,0,2020.0,\N,\N,"Music,Short"
771,Family Business: Season 2,2020,20200911,True,family business,2,True,tt12518928,video,Family Business,0,2020.0,\N,\N,"Comedy,Short"
931,Good Girls: Season 4,2021,20210307,True,good girls,4,True,tt15562706,video,Good Girls,1,2021.0,\N,\N,Adult
1002,Heist: Season 1,2021,20210714,True,heist,1,True,tt13996098,video,Heist,0,2021.0,\N,5,"Comedy,Short"
1199,Johnny Test: Season 1,2021,20210716,True,johnny test,1,True,tt15056524,video,Johnny Test,0,2021.0,\N,11,"Animation,Short"
1333,Love Is Blind: Season 1,2020,20200213,True,love is blind,1,True,tt12810302,video,Love Is Blind,0,2020.0,\N,8,"Comedy,Short"


In [29]:
netflix_data['combined_title'].value_counts()

The One: Season 1                               2
Last One Standing: Season 1                     2
Catching Killers: Season 1                      2
Secrets of Summer: Season 1                     1
Sea of Love: Season 1                           1
                                               ..
Harvey Girls Forever!: Season 5                 1
Harvey Girls Forever!: Season 4                 1
Haroun                                          1
Hard Cell: Season 1                             1
​SAINT SEIYA: Knights of the Zodiac: Part II    1
Name: combined_title, Length: 2228, dtype: int64

In [11]:
merged = pd.merge(netflix_data, part2, on=['title', '', how='left', indicator=True)

In [17]:
part2.columns

Index(['tconst', 'titleType', 'title', 'originalTitle', 'isAdult', 'startYear',
       'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')

In [21]:
type(part2['startYear'].iloc[0])

str

In [19]:
merged2 = netflix_data.merge(part2, left_on=['title', 'release_year'], right_on=['title', 'startYear'], how='left', indicator=True)

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [15]:
part2[part2['title'].str.contains('new world', na=False)]

,tconst,titleType,title,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1021248,tt10096030,tvEpisode,"new world order, part 1","New World Order, Part 1",0,2020,\N,49,"Drama,Fantasy,Horror"
1021249,tt10096032,tvEpisode,"new world order, part 2","New World Order, Part 2",0,2020,\N,49,"Drama,Fantasy,Horror"
1150884,tt10329142,tvEpisode,new world disorder,New World Disorder,0,2020,\N,\N,Action
1566222,tt11071724,tvEpisode,chapter 54: a strange new world,Chapter 54: A Strange New World,0,2020,\N,\N,"Animation,Drama"
1759328,tt11421270,tvEpisode,the new world,The New World,0,2020,\N,22,"Action,Adventure,Animation"
...,...,...,...,...,...,...,...,...,...
8943265,tt9206778,tvEpisode,seven wonders of the new world,Seven Wonders of the New World,0,2020,\N,\N,Documentary
8946057,tt9212984,tvEpisode,new world order,New World Order,0,2021,\N,47,"Action,Adventure,Drama"
9005822,tt9340846,tvSeries,zahra's new world,Zahra's New World,0,2020,\N,\N,Comedy
9006652,tt9342634,tvEpisode,a new world,A New World,0,2020,\N,\N,Comedy


In [12]:
merged['titleType'].value_counts()

tvEpisode       1078
short            817
movie            800
tvSeries         772
tvSpecial        114
tvMiniSeries      84
video             56
tvMovie           24
tvShort            4
videoGame          1
Name: titleType, dtype: int64

In [13]:
merged[merged['titleType']=='videoGame']

,combined_title,release_year,release_date,is_top10,title,season,is_tv_show,tconst,titleType,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,_merge
2249,New World: Season 1,2021,20211120,True,new world,1,True,tt11909050,videoGame,New World,0,2021,\N,\N,"Action,Adventure,Fantasy",both


In [41]:
part2[part2['originalTitle'].str.contains('3', na=False)]

,tconst,titleType,title,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
197893,tt0206169,short,the news no. 3,The News No. 3,0,2020,\N,3,Short
900347,tt0929884,tvEpisode,episode #82.30,Episode #82.30,0,2020,\N,30,Game-Show
900348,tt0929885,tvEpisode,episode #82.31,Episode #82.31,0,2020,\N,30,Game-Show
900349,tt0929886,tvEpisode,episode #82.32,Episode #82.32,0,2020,\N,30,Game-Show
900350,tt0929887,tvEpisode,episode #82.33,Episode #82.33,0,2020,\N,45,Game-Show
...,...,...,...,...,...,...,...,...,...
9231467,tt9826514,tvEpisode,episode #3.1,Episode #3.1,0,2022,\N,\N,"Action,Drama,Thriller"
9234047,tt9832026,tvEpisode,épisode 3,Épisode 3,0,2020,\N,47,Comedy
9235178,tt9834512,tvEpisode,episode #3.2,Episode #3.2,0,2020,\N,\N,Drama
9237300,tt9839038,movie,jim button and the wild 13,Jim Knopf und die Wilde 13,0,2020,\N,110,"Adventure,Comedy,Family"


In [42]:
part2[part2['tconst']=='tt4922804']

,tconst,titleType,title,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [ ]:
part2[]

In [43]:
merged[merged['_merge'] == 'left_only'].to_csv('mismatched_titles.csv', index=False)

In [53]:
merged[merged['_merge'] == 'left_only'].head()

,combined_title,release_year,release_date,is_top10,title,season,is_tv_show,tconst,titleType,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,_merge
2,10 Days With Santa Claus,2020,20201204,False,10 days with santa claus,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,100 Humans: Season 1,2020,20200313,False,100 humans,1,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
11,3%: Season 4,2020,20200814,True,3%,4,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
19,7SEEDS: Part 2,2020,20200326,True,7seeds: part 2,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
31,800 Meters: Limited Series,2022,20220325,True,800 meters: limited series,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [100]:
part2[part2['title'].str.contains('blackaf', )

,tconst,titleType,title,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [101]:
part2[part2['tconst']=='tt10311562']

,tconst,titleType,title,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1141206,tt10311562,tvSeries,#BlackAF,#BlackAF,0,2020,2020,36,Comedy
